In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.tree import export_text


ModuleNotFoundError: No module named 'sklearn'

# Win Prediction at half time - Decision Tree

## Import Dataset

In [ ]:
local_path='dataset/soccer18-19.csv'
auto='auto'
df='df_half.csv'

In [ ]:
# get current directory
path = os.getcwd()
print("Current Directory", path)
print()
 
# parent directory
parent = os.path.dirname(path)
print("Parent directory", parent)
print()

# dataset path
dataset_path=os.path.join(parent,auto,df)
print("Dataset path",dataset_path)
print()

# auto path
path_auto=os.path.join(parent,auto)
print("Dump path",path_auto)

In [ ]:
df=pd.read_csv(dataset_path)

print("shape:", df.shape)

df.head(5)